# Day 1

In [1]:
with open('day1/input.txt', 'r') as f:
    arr = f.read().splitlines()

In [2]:
arr = [int(x) for x in arr]

def two_sum(arr, x):
    arr.sort()
    l = 0
    h = len(arr) - 1
    while l < h:
        if arr[l] + arr[h] == x:
            return arr[l] * arr[h]
        elif arr[l] + arr[h] < x:
            l += 1
        else:
            h -= 1
    return -1


In [3]:
def three_sum(arr, x):
    arr.sort()
    l = 0
    h = len(arr) - 1
    for y in arr:
        tmp = two_sum(arr, x - y)
        if tmp != -1:
            return tmp * y

In [4]:
two_sum(arr, 2020)

In [5]:
three_sum(arr, 2020)

# Day 2

In [6]:
import re

# part 1
n = 0
with open('day2/input.txt', 'r') as f:
    for line in f:
        l, h, letter, pwd = re.split('[^A-Za-z0-9]+', line)[:-1]
        l, h = int(l), int(h)
        
        if len(re.findall(letter, pwd)) in range(l, h + 1):
            n += 1
            
print(f"For Part 1 there are {n} correct passwords")

In [7]:
# part 2
n = 0
with open('day2/input.txt', 'r') as f:
    for line in f:
        l, h, letter, pwd = re.split('[^A-Za-z0-9]+', line)[:-1]
        l, h = int(l), int(h)
        
        try:
            l_letter = pwd[l-1]
            h_letter = pwd[h-1]
        except IndexError:
            continue
        
        if (l_letter == letter) != (h_letter == letter):
            n += 1

In [8]:
print(f"For Part 2 there are {n} correct passwords")

# Day 3

In [9]:
import numpy as np

with open('day3/input.txt', 'r') as f:
    arr = f.read().splitlines()

In [10]:
def get_trees(arr, right, down):
    x = 0; y = 0
    xmax = len(arr[0]) 
    ymax = len(arr) - 1
    
    n_trees = 0
    while True:
        x  = (x + right) % xmax
        y += down

        if y > ymax:
            break
        if arr[y][x] == "#":
            n_trees += 1
            
    return n_trees


In [11]:
right = 3
down = 1
n_trees = get_trees(arr, right, down)
print(f"Following right {right} and down {down} we hit {n_trees} trees")

Following right 3 and down 1 we hit 193 trees


In [12]:
slopes = zip([1, 3, 5, 7, 1], [1, 1, 1, 1, 2])
trees = [get_trees(arr, x, y) for x, y in slopes]
tree_product = np.prod(trees)

In [13]:
print(f"Part two answer: {tree_product}")

Part two answer: 1355323200


# Day 4

In [1]:
import re
import numpy as np

In [21]:
# functions
def parse_line(entry):  
    return {k: v for k, v in re.findall('([a-z]{3}):([^\s(\\n)]+)', entry)}


def has_req_fields(entry):
    req_fields = set(['byr', 'iyr', 'eyr', 'hgt', 'hcl', 'ecl', 'pid'])
    missing = req_fields - set(entry.keys())
    return len(missing) == 0


def int_in_range(x, low, high):
    return x.isdigit() and int(x) >= low and int(x) <= high


def validate_field(field, value):
    # expects field (str) and value (str), return True if valid, False otherwise
    if field == 'byr':
        return int_in_range(value, 1920, 2002)

    if field == 'iyr':
        return int_in_range(value, 2010, 2020)
    
    if field == 'eyr':
        return int_in_range(value, 2020, 2030)

    if field == 'hgt':
        if not re.fullmatch('\d+(cm|in)', value):
            return False
        digit = int(re.findall('\d+', value)[0])
        unit = re.findall('cm|in', value)[0]
        if unit == 'cm':
            return digit in range(150, 194)
        if unit == 'in':
            return digit in range(59, 77)

    if field == 'hcl':
        return bool(re.fullmatch('#[a-f0-9]{6}', value))

    if field == 'ecl':
        return value in ['amb', 'blu', 'brn', 'gry', 'grn', 'hzl', 'oth']

    if field == 'pid':
        return bool(re.fullmatch('\d{9}', value))
    
def all_fields_correct(entry):
    for field in entry:
        # ignore country id field
        if field == 'cid':
            continue
        if not validate_field(field, entry[field]):
            return False
    return True

In [22]:
# read data
with open('day4/input.txt', 'r') as f:
    raw = f.read().split('\n\n')
entries = [parse_line(x) for x in raw]

# part 1
n_valid = 0
for entry in entries:
    if has_req_fields(entry):
        n_valid += 1

print(f"Part 1: {n_valid} valid fields")

Part 1: 247 valid fields


In [23]:
# part 2
n_valid = 0
for entry in entries:
    if has_req_fields(entry) and all_fields_correct(entry):
        n_valid += 1
        
print(f"Part 2: {n_valid} valid fields")

Part 2: 145 valid fields


# Day 5

In [74]:
from math import floor, ceil
import numpy as np

with open('day5/input.txt', 'r') as f:
    arr = f.read().splitlines()

In [104]:
def get_pos(s, nmax):
    # takes s (str) the binary partition string, and nmax (int) the max number of seats
    low = 0
    high = nmax - 1
    for i in s:
        if i in ('F', 'L'):
            high = floor((low + high) / 2)
        elif i in ('B', 'R'):
            low = ceil((low + high) / 2)
    return low

def get_seatid(row, col):
    return (row * 8) + col

In [105]:
row_max = 128
col_max = 8
seatids = []
filled_seats = np.full((row_max, col_max), 0)

# part 1
for seat in arr:
    # get the row and column
    row, col = get_pos(seat[:7], row_max), get_pos(seat[-3:], col_max)
    
    # update arrays
    filled_seats[row, col] = 1
    seatids.append(get_seatid(row, col))

In [106]:
print(f"Max seat ID: {max(seatids)}")

Max seat ID: 801


In [112]:
# part 2
for row, col in np.argwhere(filled_seats == 0):
    seatid = get_seatid(row, col)
    if seatid - 1 in seatids and seatid + 1 in seatids:
        break
        
print(f"My seat: ({row}, {col}), seatid = {get_seatid(row, col)}")

My seat: (74, 5), seatid = 597


# Day 6

In [8]:
import re

with open('day6/input.txt', 'r') as f:
    arr = f.read().split('\n\n')

In [39]:
n_yes = 0
for group in arr:
    letters = set(re.findall('[a-z]', group))
    n_yes += len(letters)

print(f"Part 1: {n_yes}")

Part 1: 6565


In [68]:
n_yes = 0
for group in arr:
    answers = [set(x) for x in group.rstrip().split('\n')]
    common = set.intersection(*answers)
    n_yes += len(common)

In [69]:
print(f"Part 2: {n_yes}")

Part 2: 3137
